In [6]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import librosa
import librosa.display

import plotly.express as px
import IPython.display as ipd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tqdm import tqdm, trange
from librosa import feature, amplitude_to_db, load

from tqdm.auto import tqdm
from plotly.subplots import make_subplots

from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout

pd.plotting.register_matplotlib_converters()

%matplotlib inline

In [7]:
data_path = 'C:\\Users\\sojou\\archive\\UrbanSound8K.csv'
df = pd.read_csv(data_path)
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [8]:
sample_path = []
path = 'C:\\Users\\sojou\\archive\\'

for index_num,row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(path),'fold'+str(row["fold"])+'/',str(row['slice_file_name']))    
    sample_path.append(file_name)
df['path'] = sample_path

0it [00:00, ?it/s]

In [9]:
count1 = (df['class'] == "car_horn").sum()
count3 = (df['class'] == "dog_bark").sum()
count8 = (df['class'] == "siren").sum()
print(count1,count3,count8)
#클래스1,3,8인 경우의 데이터 개수

429 1000 929


In [10]:
target_count = count1

car_horn_data = df[df['class'] == "car_horn"]
dog_bark_data = df[df['class'] == "dog_bark"]
siren_data = df[df['class'] == "siren"]

car_horn_sampled = car_horn_data.sample(n=target_count, random_state=42)
dog_bark_sampled = dog_bark_data.sample(n=target_count, random_state=42)
siren_sampled = siren_data.sample(n=target_count, random_state=42)

# 클래스 1의 데이터와 샘플링한 클래스 3과 8의 데이터를 합치기
df1 = pd.concat([car_horn_sampled, dog_bark_sampled, siren_sampled])
df1

,slice_file_name,fsID,start,end,salience,fold,classID,class,path
8724,99812-1-0-1.wav,99812,38.676254,42.676254,2,7,1,car_horn,C:\Users\sojou\archive\fold7/99812-1-0-1.wav
3440,17074-1-0-0.wav,17074,3.540527,4.403714,2,10,1,car_horn,C:\Users\sojou\archive\fold10/17074-1-0-0.wav
5149,199769-1-0-6.wav,199769,3.848273,7.848273,1,3,1,car_horn,C:\Users\sojou\archive\fold3/199769-1-0-6.wav
1311,132073-1-4-0.wav,132073,8.310243,9.094228,2,2,1,car_horn,C:\Users\sojou\archive\fold2/132073-1-4-0.wav
7602,72567-1-4-7.wav,72567,58.180670,62.180670,2,2,1,car_horn,C:\Users\sojou\archive\fold2/72567-1-4-7.wav
...,...,...,...,...,...,...,...,...,...
5773,24347-8-0-27.wav,24347,17.620371,21.620371,2,4,8,siren,C:\Users\sojou\archive\fold4/24347-8-0-27.wav
6409,42117-8-0-0.wav,42117,88.660252,92.660252,2,3,8,siren,C:\Users\sojou\archive\fold3/42117-8-0-0.wav
5780,24347-8-0-33.wav,24347,20.620371,24.620371,2,4,8,siren,C:\Users\sojou\archive\fold4/24347-8-0-33.wav
2675,159745-8-1-3.wav,159745,3.199906,7.199906,2,9,8,siren,C:\Users\sojou\archive\fold9/159745-8-1-3.wav


In [11]:
def features_extract(file):
    sample,sample_rate = librosa.load(file_name,res_type='kaiser_fast')
    feature = librosa.feature.mfcc(y=sample,sr=sample_rate,n_mfcc=50)
    scaled_feature = np.mean(feature.T,axis=0)
    return scaled_feature

extracted = []
path = 'C:\\Users\\sojou\\archive'

for index_num,row in tqdm(df.iterrows()):
    file_name = os.path.join(os.path.abspath(path),'fold'+str(row["fold"])+'/',str(row['slice_file_name'])) 
    final_class_labels = row['class']   
    data= features_extract(file_name)    
    extracted.append([data,final_class_labels])

0it [00:00, ?it/s]

ModuleNotFoundError: No module named 'resampy'

This error is lazily reported, having originally occured in
  File C:\Users\sojou\PycharmProjects\Pronobo\.venv\Lib\site-packages\librosa\core\audio.py, line 33, in <module>

----> resampy = lazy.load("resampy")

In [ ]:
ext_df = pd.DataFrame(extracted,columns=['feature','class'])
ext_df1=ext_df[ext_df['class'].isin(["car_horn","dog_bark","siren"])]
ext_df1

In [17]:
x = np.array(ext_df1['feature'].tolist())
y = np.array(ext_df1['class'].tolist())

In [18]:
le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

ValueError: zero-size array to reduction operation maximum which has no identity